# NewAIBench Experiment Runner Tutorial

Notebook này sẽ hướng dẫn bạn từng bước để chạy experiment với NewAIBench framework, từ việc pull git repository đến khi hoàn thành experiment.

## Tổng quan
- Clone repository và cập nhật code mới nhất
- Cài đặt dependencies
- Tạo custom YAML configuration
- Chạy experiment và phân tích kết quả

⚠️ **Lưu ý**: Notebook này được thiết kế để chạy trong environment đã có Python và Git.

In [ ]:
import os
import sys
import subprocess
import yaml
import json
from pathlib import Path
import datetime
from IPython.display import Markdown, display

# Thiết lập working directory
WORK_DIR = "/home/hkduy/NewAI/new_bench"
os.chdir(WORK_DIR)
print(f"Working directory: {os.getcwd()}")

## 1. Clone Repository và Cập nhật Code

Đầu tiên, chúng ta sẽ clone repository (nếu chưa có) hoặc pull code mới nhất từ remote repository.

In [ ]:
# Kiểm tra xem đã có git repository chưa
if os.path.exists('.git'):
    print("✅ Git repository đã tồn tại")
    # Pull latest changes
    try:
        result = subprocess.run(['git', 'status', '--porcelain'], 
                              capture_output=True, text=True, check=True)
        if result.stdout.strip():
            print("⚠️ Có changes chưa commit:")
            print(result.stdout)
            print("\nBạn có thể stash changes trước khi pull:")
            print("git stash")
        else:
            print("Working directory clean, pulling latest changes...")
            pull_result = subprocess.run(['git', 'pull'], 
                                       capture_output=True, text=True, check=True)
            print(f"✅ Git pull completed: {pull_result.stdout}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Git operation failed: {e}")
else:
    print("❌ Không tìm thấy git repository")
    print("Nếu bạn cần clone repository, uncomment và chạy lệnh dưới đây:")
    print("# git clone <repository_url> .")

In [ ]:
# Hiển thị thông tin branch hiện tại
try:
    branch_result = subprocess.run(['git', 'branch', '--show-current'], 
                                 capture_output=True, text=True, check=True)
    current_branch = branch_result.stdout.strip()
    print(f"🔄 Current branch: {current_branch}")
    
    # Hiển thị commit gần nhất
    commit_result = subprocess.run(['git', 'log', '-1', '--oneline'], 
                                 capture_output=True, text=True, check=True)
    latest_commit = commit_result.stdout.strip()
    print(f"📝 Latest commit: {latest_commit}")
except subprocess.CalledProcessError as e:
    print(f"❌ Cannot get git info: {e}")

## 2. Cài đặt Dependencies

Cài đặt tất cả các dependencies cần thiết cho NewAIBench framework.

In [ ]:
# Kiểm tra Python version
print(f"🐍 Python version: {sys.version}")
print(f"📂 Python executable: {sys.executable}")

# Kiểm tra xem có requirements.txt không
if os.path.exists('requirements.txt'):
    print("✅ Found requirements.txt")
    with open('requirements.txt', 'r') as f:
        requirements = f.read()
    print("\n📋 Requirements:")
    print(requirements[:500] + "..." if len(requirements) > 500 else requirements)
else:
    print("❌ requirements.txt not found")

In [ ]:
# Cài đặt dependencies
print("🔧 Installing dependencies...")
try:
    # Upgrade pip first
    subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'], 
                  check=True)
    print("✅ Pip upgraded")
    
    # Install requirements
    if os.path.exists('requirements.txt'):
        result = subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], 
                              capture_output=True, text=True, check=True)
        print("✅ Requirements installed successfully")
        if result.stderr:
            print(f"⚠️ Warnings: {result.stderr[:200]}...")
    
    # Install package in development mode
    if os.path.exists('setup.py'):
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '.'], 
                      check=True)
        print("✅ Package installed in development mode")
        
except subprocess.CalledProcessError as e:
    print(f"❌ Installation failed: {e}")
    print(f"Error output: {e.stderr if hasattr(e, 'stderr') else 'No error details'}")

In [ ]:
# Kiểm tra xem NewAIBench đã được cài đặt chưa
try:
    # Add src to path if needed
    src_path = os.path.join(os.getcwd(), "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
    
    # Test import
    from newaibench.experiment import ExperimentRunner, ExperimentConfig
    print("✅ NewAIBench import successful")
    
    # Check if run_experiment.py exists
    if os.path.exists('run_experiment.py'):
        print("✅ run_experiment.py found")
    else:
        print("❌ run_experiment.py not found")
        
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Có thể cần cài đặt thêm dependencies hoặc kiểm tra PYTHONPATH")

## 3. Tạo Custom YAML Configuration

Bây giờ chúng ta sẽ tạo một file cấu hình YAML tùy chỉnh cho experiment của bạn.

### 3.1 Xem các template có sẵn

In [ ]:
# Liệt kê các experiment templates có sẵn
print("📂 Existing experiment configurations:")
print("\n1. In examples/experiments/:")
examples_exp_dir = Path("examples/experiments")
if examples_exp_dir.exists():
    for yaml_file in examples_exp_dir.glob("*.yaml"):
        print(f"   - {yaml_file.name}")
    for json_file in examples_exp_dir.glob("*.json"):
        print(f"   - {json_file.name}")
else:
    print("   (directory not found)")

print("\n2. In experiments/:")
exp_dir = Path("experiments")
if exp_dir.exists():
    for yaml_file in exp_dir.glob("*.yaml"):
        print(f"   - {yaml_file.name}")
    for json_file in exp_dir.glob("*.json"):
        print(f"   - {json_file.name}")
else:
    print("   (directory not found)")

In [ ]:
# Tạo template mẫu bằng CLI tool
print("🔧 Creating a basic template...")
try:
    result = subprocess.run([sys.executable, 'run_experiment.py', 
                           '--create-template', 'basic', 
                           '--output-config', 'my_custom_experiment.yaml'], 
                          capture_output=True, text=True, check=True)
    print("✅ Template created successfully")
    print(f"Output: {result.stdout}")
except subprocess.CalledProcessError as e:
    print(f"❌ Template creation failed: {e}")
    print(f"Error: {e.stderr}")

In [ ]:
# Hiển thị nội dung template vừa tạo
if os.path.exists('my_custom_experiment.yaml'):
    print("📝 Generated template content:")
    with open('my_custom_experiment.yaml', 'r') as f:
        template_content = f.read()
    print(template_content)
else:
    print("❌ Template file not found")

### 3.2 Tùy chỉnh Configuration

Bây giờ chúng ta sẽ tùy chỉnh file YAML theo nhu cầu của bạn. Bạn có thể chỉnh sửa trực tiếp file hoặc tạo một file mới.

In [ ]:
# Tạo custom YAML configuration
custom_config = {
    'description': 'Custom experiment for testing NewAIBench',
    'models': [
        {
            'name': 'bm25_model',
            'type': 'sparse',
            'model_name_or_path': '',  # BM25 không cần model path
            'parameters': {
                'k1': 1.2,
                'b': 0.75
            },
            'device': 'cpu',
            'batch_size': 32
        }
    ],
    'datasets': [
        {
            'name': 'test_dataset',
            'type': 'text',
            'data_dir': './BKAI_law_data/newaibench_formatted_data/legal_data',  # Sử dụng dataset có sẵn
            'config_overrides': {
                'cache_enabled': True
            }
        }
    ],
    'evaluation': {
        'metrics': ['ndcg', 'map', 'recall'],
        'k_values': [1, 5, 10],
        'top_k': 100,
        'save_run_file': True
    },
    'output': {
        'output_dir': './results',
        'experiment_name': f'custom_experiment_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}',
        'log_level': 'INFO',
        'overwrite': True
    }
}

# Lưu vào file YAML
with open('my_custom_experiment.yaml', 'w') as f:
    yaml.dump(custom_config, f, default_flow_style=False, sort_keys=False)

print("✅ Custom YAML configuration created:")
with open('my_custom_experiment.yaml', 'r') as f:
    print(f.read())

In [ ]:
# Validate configuration file
print("🔍 Validating configuration...")
try:
    # Test loading configuration
    from newaibench.experiment import load_experiment_config
    config = load_experiment_config('my_custom_experiment.yaml')
    print("✅ Configuration file is valid")
    
    # Show summary
    print(f"\n📊 Configuration Summary:")
    print(f"   - Models: {len(config.models)}")
    print(f"   - Datasets: {len(config.datasets)}")
    print(f"   - Metrics: {', '.join(config.evaluation.metrics)}")
    print(f"   - Output: {config.output.output_dir}")
    
except Exception as e:
    print(f"❌ Configuration validation failed: {e}")
    print("Vui lòng kiểm tra lại file YAML")

### 3.3 Kiểm tra Dataset

Trước khi chạy experiment, hãy kiểm tra xem dataset có tồn tại và đúng format không.

In [ ]:
# Kiểm tra dataset
dataset_path = './BKAI_law_data/newaibench_formatted_data/legal_data'
print(f"🔍 Checking dataset at: {dataset_path}")

if os.path.exists(dataset_path):
    print("✅ Dataset directory exists")
    
    # List contents
    contents = list(Path(dataset_path).iterdir())
    print(f"📁 Dataset contains {len(contents)} items:")
    for item in contents[:10]:  # Show first 10 items
        print(f"   - {item.name}")
    if len(contents) > 10:
        print(f"   ... and {len(contents) - 10} more items")
    
    # Check for required files
    required_files = ['corpus.jsonl', 'queries.jsonl', 'qrels.jsonl']
    for req_file in required_files:
        file_path = Path(dataset_path) / req_file
        if file_path.exists():
            print(f"✅ Found {req_file} ({file_path.stat().st_size} bytes)")
        else:
            print(f"❌ Missing {req_file}")
else:
    print("❌ Dataset directory not found")
    print("Available datasets in BKAI_law_data/:")
    if os.path.exists('./BKAI_law_data'):
        for item in Path('./BKAI_law_data').iterdir():
            print(f"   - {item.name}")

## 4. Chạy Experiment

Bây giờ chúng ta sẽ chạy experiment với cấu hình đã tạo.

In [ ]:
# Chạy dry-run trước để kiểm tra
print("🧪 Running dry-run to validate setup...")
try:
    result = subprocess.run([sys.executable, 'run_experiment.py', 
                           '--config', 'my_custom_experiment.yaml',
                           '--dry-run'], 
                          capture_output=True, text=True, check=True)
    print("✅ Dry-run completed successfully")
    print("📋 Execution plan:")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"❌ Dry-run failed: {e}")
    print(f"Error output: {e.stderr}")
    print("\nVui lòng kiểm tra lại configuration và dataset")

In [ ]:
# Chạy experiment thực tế
print("🚀 Starting actual experiment...")
print("⚠️ Experiment này có thể mất vài phút để hoàn thành")
print("\nNếu bạn muốn chạy experiment, uncomment dòng code dưới đây:")
print("\n# Chạy experiment (uncomment để thực thi)")
run_experiment = False  # Đặt thành True nếu muốn chạy ngay

if run_experiment:
    try:
        # Tạo process để chạy experiment
        process = subprocess.Popen([sys.executable, 'run_experiment.py', 
                                  '--config', 'my_custom_experiment.yaml'],
                                 stdout=subprocess.PIPE,
                                 stderr=subprocess.PIPE,
                                 text=True,
                                 bufsize=1,
                                 universal_newlines=True)
        
        print("Experiment đang chạy...")
        
        # Real-time output
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                print(output.strip())
        
        # Get final result
        return_code = process.poll()
        stderr_output = process.stderr.read()
        
        if return_code == 0:
            print("\n✅ Experiment completed successfully!")
        else:
            print(f"\n❌ Experiment failed with return code {return_code}")
            if stderr_output:
                print(f"Error: {stderr_output}")
                
    except Exception as e:
        print(f"❌ Failed to run experiment: {e}")
else:
    print("\nĐể chạy experiment, bạn có thể:")
    print("1. Đặt run_experiment = True ở cell trên")
    print("2. Hoặc chạy command line: python run_experiment.py --config my_custom_experiment.yaml")

### 4.1 Chạy Experiment Manually (Tùy chọn)

Nếu bạn muốn chạy experiment từ command line hoặc có kiểm soát nhiều hơn:

In [ ]:
# Hiển thị command để chạy experiment
config_file = 'my_custom_experiment.yaml'
command = f"python run_experiment.py --config {config_file}"

print("🔧 Command để chạy experiment:")
print(f"cd {os.getcwd()}")
print(command)
print()
print("📋 Các options khác:")
print(f"# Chạy dry-run: python run_experiment.py --config {config_file} --dry-run")
print(f"# Với log debug: python run_experiment.py --config {config_file} --log-level DEBUG")
print(f"# Overwrite results: python run_experiment.py --config {config_file} --overwrite")

# Copy command to clipboard (if possible)
try:
    import pyperclip
    pyperclip.copy(command)
    print("\n✅ Command copied to clipboard!")
except ImportError:
    print("\n💡 Tip: Install pyperclip để auto-copy command: pip install pyperclip")

## 5. Kiểm tra Kết quả

Sau khi experiment hoàn thành, chúng ta sẽ kiểm tra kết quả.

In [ ]:
# Kiểm tra kết quả experiment
results_dir = './results'
print(f"🔍 Checking results in: {results_dir}")

if os.path.exists(results_dir):
    print("✅ Results directory exists")
    
    # List experiment directories
    experiments = [d for d in Path(results_dir).iterdir() if d.is_dir()]
    print(f"\n📊 Found {len(experiments)} experiment(s):")
    
    for exp_dir in sorted(experiments, key=lambda x: x.stat().st_mtime, reverse=True):
        print(f"\n📁 {exp_dir.name}:")
        contents = list(exp_dir.iterdir())
        for item in contents:
            if item.is_file():
                size = item.stat().st_size
                print(f"   📄 {item.name} ({size} bytes)")
            else:
                print(f"   📁 {item.name}/")
        
        # Check for evaluation results
        eval_file = exp_dir / 'evaluation_results.json'
        if eval_file.exists():
            print(f"   ✅ Found evaluation results")
            try:
                with open(eval_file, 'r') as f:
                    eval_data = json.load(f)
                print(f"   📈 {len(eval_data)} result(s) available")
            except Exception as e:
                print(f"   ❌ Could not read evaluation results: {e}")
else:
    print("❌ Results directory not found")
    print("Experiment chưa được chạy hoặc chưa hoàn thành")

In [ ]:
# Load và hiển thị kết quả chi tiết (nếu có)
def display_evaluation_results(results_dir):
    experiments = [d for d in Path(results_dir).iterdir() if d.is_dir()]
    
    if not experiments:
        print("❌ No experiment results found")
        return
    
    # Get latest experiment
    latest_exp = max(experiments, key=lambda x: x.stat().st_mtime)
    print(f"📊 Latest experiment: {latest_exp.name}")
    
    eval_file = latest_exp / 'evaluation_results.json'
    if eval_file.exists():
        try:
            with open(eval_file, 'r') as f:
                results = json.load(f)
            
            print("\n📈 Evaluation Results:")
            print("=" * 50)
            
            for result in results:
                model_name = result.get('model', 'Unknown')
                dataset_name = result.get('dataset', 'Unknown')
                metrics = result.get('metrics', {})
                
                print(f"\n🤖 Model: {model_name}")
                print(f"📚 Dataset: {dataset_name}")
                print(f"📊 Metrics:")
                
                for metric_name, values in metrics.items():
                    print(f"   {metric_name}:")
                    if isinstance(values, dict):
                        for k, v in values.items():
                            print(f"     @{k}: {v:.4f}")
                    else:
                        print(f"     {values:.4f}")
                        
        except Exception as e:
            print(f"❌ Error reading results: {e}")
    else:
        print("❌ No evaluation_results.json found")

if os.path.exists(results_dir):
    display_evaluation_results(results_dir)
else:
    print("Chưa có kết quả để hiển thị")

## 6. Tóm tắt và Bước tiếp theo

🎉 **Chúc mừng!** Bạn đã hoàn thành tutorial sử dụng NewAIBench.

### Những gì đã làm:
1. ✅ Clone/update repository
2. ✅ Cài đặt dependencies
3. ✅ Tạo custom YAML configuration
4. ✅ Chạy experiment (hoặc chuẩn bị để chạy)
5. ✅ Kiểm tra kết quả

### Bước tiếp theo:
1. **Tùy chỉnh thêm**: Chỉnh sửa file YAML để thử nghiệm với các models và datasets khác
2. **Phân tích kết quả**: Sử dụng các tools có sẵn để phân tích kết quả chi tiết
3. **Chạy nhiều experiments**: So sánh hiệu suất của các models khác nhau
4. **Tối ưu hóa**: Điều chỉnh parameters để cải thiện kết quả

### Tài liệu tham khảo:
- 📚 **Documentation**: `docs/` directory
- 🧪 **Examples**: `examples/` directory  
- ⚙️ **Configuration**: `examples/experiments/` directory

### Lưu ý quan trọng:
- ⚠️ Luôn chạy `--dry-run` trước khi chạy experiment thực tế
- 💾 Backup kết quả quan trọng
- 🔧 Kiểm tra logs nếu có lỗi
- 📊 So sánh kết quả từ nhiều experiments khác nhau

In [ ]:
# 🛠️ Utility functions để sử dụng sau này

def quick_experiment(model_type, dataset_path, experiment_name=None):
    """Tạo và chạy experiment nhanh"""
    if not experiment_name:
        experiment_name = f"quick_{model_type}_{datetime.datetime.now().strftime('%H%M%S')}"
    
    config = {
        'description': f'Quick {model_type} experiment',
        'models': [{
            'name': f'{model_type}_model',
            'type': model_type,
            'model_name_or_path': '',
            'device': 'auto'
        }],
        'datasets': [{
            'name': 'dataset',
            'type': 'text',
            'data_dir': dataset_path
        }],
        'evaluation': {
            'metrics': ['ndcg', 'map'],
            'k_values': [1, 5, 10],
            'top_k': 100
        },
        'output': {
            'output_dir': './results',
            'experiment_name': experiment_name,
            'log_level': 'INFO'
        }
    }
    
    config_file = f'{experiment_name}.yaml'
    with open(config_file, 'w') as f:
        yaml.dump(config, f, default_flow_style=False)
    
    print(f"✅ Created config: {config_file}")
    print(f"▶️ To run: python run_experiment.py --config {config_file}")
    return config_file

def list_recent_experiments(n=5):
    """Liệt kê các experiment gần đây"""
    if not os.path.exists('./results'):
        print("❌ No results directory found")
        return
    
    experiments = [d for d in Path('./results').iterdir() if d.is_dir()]
    experiments.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    
    print(f"📊 {min(n, len(experiments))} most recent experiments:")
    for i, exp in enumerate(experiments[:n]):
        mtime = datetime.datetime.fromtimestamp(exp.stat().st_mtime)
        print(f"  {i+1}. {exp.name} ({mtime.strftime('%Y-%m-%d %H:%M')})")

print("🔧 Utility functions loaded:")
print("  - quick_experiment(model_type, dataset_path, experiment_name=None)")
print("  - list_recent_experiments(n=5)")
print("\nVí dụ:")
print("  quick_experiment('sparse', './BKAI_law_data/newaibench_formatted_data/legal_data')")
print("  list_recent_experiments()")